In [1]:
### Comment Lines and useful info, commands

#grp_inx = list(df[df['make_model'] == 'Audi A3']['Body'].index)
#df[df["make_model"] == "Audi A3"]["body_type"].mode() ### Audi a3 lerdeki en cok body type.
#oto[col_name].iloc[grp_inx] = oto[col_name].iloc[grp_inx].fillna()
##############################################################################################
#[i for i in df.columns if df[i].notna().mean() < 0.5]

##############################################################################################

# in future i will update here to drop columns
# for i in df.columns:
#     if df.index.stop == df[i].isnull().sum():
#         print("------------------------->",i,df[i].isnull().sum())
#     print(i,df[i].isnull().sum())
##############################################################################################
# to see data in desired counts 
#pd.options.display.max_columns = 15
#pd.options.display.max_rows = 6
#############################################################################################
#df[["price"]].sort_values(by="price",axis=0)
#############################################################################################
#Delete rows that price is lower than 500Euros
#df.drop(df[df["price"]<500].index,inplace=True)
#############################################################################################
#df[["price"]].dropna()
#############################################################################################
#sns.boxplot(x='price',data = df)
#############################################################################################
#get_df_wo_outlier(df,"price")
#############################################################################################
#df.columns
#############################################################################################
#fix values at rows by deleting "\n" in it.
#dfdeneme  = df.replace("\n","")
#############################################################################################
#to have a quick look at data, which columns contain what and how many.
#for i in df.columns:
    #print(df[i].value_counts().rename_axis(i +' unique_values').reset_index(name=i +' counts'))
#############################################################################################    
# important_columns = ['make_model',
# 'km',
# 'registration',
# 'Type',
# 'Body',
# 'Gearing Type',
# 'Drive chain',
# 'Fuel',
# 'Consumption',
# 'Comfort & Convenience',
# 'Entertainment & Media',
# 'Extras',
# 'Safety & Security',
# 'Gears',
# 'Country version',
# 'Last Service Date']
#df[important_columns]
#############################################################################################
# if str(df.at[2765, 'Type'] == 'nan'):
#     print("sss")
# else:
#     print("aaa")
#############################################################################################
#get_all_df(df)
#df = df.astype({"Country version" : "str"})
#############################################################################################
#Convert list type columns to new column with list`s elements
# df = list_to_columns_add2(df,"Full Service") #### NO need to convert NAN to List..
# df = list_to_columns_add2(df,"Consumption") #### NO need to convert NAN to List..
# df = list_to_columns_add2(df,"Type") #### NO need to convert NAN to List..
#df  = list_to_columns_add2(df,"Country version")
#############################################################################################
#Nan_to_list(df,"Country version")
#df  = list_to_columns_add2(df,"Country version")
#df[["Country version"]]
#df[["Country version"]] = df[["Country version"]].replace('\\n',np.NaN, regex=True)
#df[["Country version"]]
#############################################################################################
#for i in df.columns:
    #print(i)
#############################################################################################
#print( [i for i in df["Body"][0] if i!='\n'] )
#############################################################################################
#list_of_columns
#non_list_of_columns
#############################################################################################
#pd.options.display.max_columns = 14
#pd.options.display.max_rows = 5
#df.columns.to_list()

# Functions

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import zscore
import numpy as np
import itertools

import warnings
warnings.filterwarnings("ignore")

In [3]:
#Functions :
#convert lists values to new columns with this no need to change column type to string or NAN to [] values normally requires before applying below.
def list_to_columns(df,col): 
    dftemp = pd.DataFrame([pd.Series(x) for x in df[col]]) # that is better and quicker. no need to run NAN_to_list.?
    column_names= []
    max_element = len(max(df.loc[df[col].apply(lambda x : isinstance(x, list))][col], key=len))
    #max_element = len(max(df.loc[df[col].notna()][col], key=len))
    for i in range(max_element):
        column_names.append(col + str(i+1))
    dftemp.columns = column_names
    df = df.drop(col,axis=1)
    if max_element == 1:
        dftemp.rename(columns={col+'1': col},inplace = True)
    df_added = pd.concat([df, dftemp], axis=1, sort=False)
    return df_added

#Convert list of features as new columns and check if that feature exist on that car or not.
def list_to_feat_cols(df,col_list):
    dftemp = pd.DataFrame()
    for col in col_list:
        #max_list = max(df.loc[df[col].apply(lambda x : isinstance(x, list))][col], key=len)
        a = df[col].explode().unique()
        max_list = sorted([i for i in a if not pd.isnull(list(a)[list(a).index(i)])])
        for i in range(len(max_list)):
            dftemp[f"{col} {str(i+1)} {max_list[i]}"] = df[col].apply(lambda x : True if (type(x) == list and max_list[i] in x) \
               else (False if (type(x) == list and max_list[i] not in x) else x ))
        df = df.drop(col,axis=1)
    df_added = pd.concat([df, dftemp], axis=1, sort=False)
    return df_added


# check a column if it contains list of values. returns 2 list that we need to define
def check_is_list(df):
    list_of_columns = []
    non_list_of_columns = []
    for i in df.columns:
        counter_list=0
        counter_non_list=0
        for j in df[i].values:
            if isinstance(j, list):
                counter_list+=1
            else:
                counter_non_list+=1
        if counter_list > 0:
            list_of_columns.append(i)
        else :
            non_list_of_columns.append(i)
    return list_of_columns,non_list_of_columns
#changes a phrase in a list to another value
def del_str_phrase(phrase,liste):
    while(phrase in liste) : 
        liste.remove(phrase)
        
#get the df colon without outliers specified
def get_df_wo_outlier(df,col,range=1.5):
    Q1 = df[col].quantile(q=0.25)
    Q3 = df[col].quantile(q=0.75)
    IQR = Q3-Q1
    return df[col][~((df[col] <  Q1-(range*IQR)) |(df[col] > Q3+(range*IQR))) ]

#to have a look at data, which columns contain what and how many.
def get_all_df(df):
    for i in df.columns:
        print(df[i].value_counts(dropna = False).rename_axis(i +' unique_values').reset_index(name=i +' counts'))

#quick look on a column        
def get_col_details(df,col):
    print("VALUE COUNTS")
    print(df[col].value_counts(dropna=False))
    print("-----------------------\n","UNIQUE VALUES")
    print(df[col].unique())
    print("-----------------------\n","NULL SUM")
    print(df[col].isnull().sum())
    print("-----------------------\n","NULL Percentage")
    print(df[col].isnull().sum()*100 /df[col].index.stop )
    print("-----------------------\n","COLUMN with NONE-NULL-NAN")
    print(df.query(f'{col} != {col}')[col])
    
#updates values NAN to empty list : []
def Nan_to_list(df,col):
    Nan_rows = df[col].isnull()
    df.loc[Nan_rows,col] = df.loc[Nan_rows,col].apply(lambda x : [])
    
def outlier_zscore(df, col, min_z=1, max_z = 5, step = 0.1, print_list = False):
    z_scores = zscore(df[col].dropna())
    threshold_list = []
    for threshold in np.arange(min_z, max_z, step):
        threshold_list.append((threshold, len(np.where(z_scores > threshold)[0])))
        df_outlier = pd.DataFrame(threshold_list, columns = ['threshold', 'outlier_count'])
        df_outlier['pct'] = (df_outlier.outlier_count - df_outlier.outlier_count.shift(-1))/df_outlier.outlier_count*100
    plt.plot(df_outlier.threshold, df_outlier.outlier_count)
    best_treshold = round(df_outlier.iloc[df_outlier.pct.argmax(), 0],2)
    outlier_limit = int(df[col].dropna().mean() + (df[col].dropna().std()) * df_outlier.iloc[df_outlier.pct.argmax(), 0])
    percentile_threshold = stats.percentileofscore(df[col].dropna(), outlier_limit)
    plt.vlines(best_treshold, 0, df_outlier.outlier_count.max(), 
               colors="r", ls = ":"
              )
    plt.annotate("Zscore : {}\nValue : {}\nPercentile : {}".format(best_treshold, outlier_limit, 
                                                                   (np.round(percentile_threshold, 3), 
                                                                    np.round(100-percentile_threshold, 3))), 
                 (best_treshold, df_outlier.outlier_count.max()/2))
    #plt.show()
    if print_list:
        print(df_outlier)
    return (plt, df_outlier, best_treshold, outlier_limit, percentile)

def outlier_inspect(df, col, min_z=1, max_z = 5, step = 0.5, max_hist = None, bins = 50):
    fig = plt.figure(figsize=(20, 6))
    fig.suptitle(col, fontsize=16)
    plt.subplot(1,3,1)
    if max_hist == None:
        sns.distplot(df[col], kde=False, bins = 50)
    else :
        sns.distplot(df[df[col]<=max_hist][col], kde=False, bins = 50)
    plt.subplot(1,3,2)
    sns.boxplot(df[col])
    plt.subplot(1,3,3)
    z_score_inspect = outlier_zscore(df, col, min_z=min_z, max_z = max_z, step = step)
    plt.show()

# Entrance

In [4]:
# read json file with lines method.
df = pd.read_json("scout_car.json", lines=True)

In [5]:
#lets see all column and value_counts
#get_all_df(df)

In [6]:
#fix column name by deleting "\n" in it.
df.rename(columns=lambda x: x.replace("\n",""),inplace=True)

In [7]:
pd.options.display.max_columns = 600
pd.options.display.max_rows = 6
df.head(2)

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,kW,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,Offer Number,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Model Code,Gearing Type,Displacement,Cylinders,Weight,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,Comfort & Convenience,Entertainment & Media,Extras,Safety & Security,description,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n2\n,"[\n06/2021\n, \n99 g CO2/km (comb)\n]","[\nYes\n, \nEuro 6\n]","[\n, \n, \n4 (Green)\n]","[\n, \n]","[\n, \n]",[],\nAudi\n,"[\n, A1, \n]",[\nLR-062483\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nMythosschwarz\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n5\n],[\n5\n],[\n0588/BDF\n],"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],"[\n1,220 kg\n]",[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Armrest, Automatic climate ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Catalytic Converter, Voice Cont...","[ABS, Central door lock, Daytime running light...","[\n, Sicherheit:, , Deaktivierung für Beifahr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,NaN,141 kW,"[, Used, , Gasoline]",NaN,NaN,NaN,NaN,NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",NaN,"[\n, 2017, \n]","[\n, Red, \n]",NaN,NaN,"[\nCloth, Grey\n]","[\n, Sedans, \n]",[\n3\n],[\n4\n],[\n0588/BCY\n],"[\n, Automatic, \n]","[\n1,798 cc\n]",[\n4\n],"[\n1,255 kg\n]",[\nfront\n],"[\n, Gasoline, \n]","[[5.6 l/100 km (comb)], [7.1 l/100 km (city)],...",[\n129 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Automatic climate control, ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Sport seats, Sport suspension, ...","[ABS, Central door lock, Central door lock wit...",[\nLangstreckenfahrzeug daher die hohe Kilomet...,[\n4 (Green)\n],[\n7\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#drop unnecessary columns #subjectif decision

#no need = 'url', 'description','Offer Number'
#same info already exist = "make_model","Body2", "prev_owner"
unnecassary_columns = ['url', 'description', 'Offer Number',"prev_owner","Model Code","Weight","Make","Model"]
for i in unnecassary_columns:
    try:
        df.drop(i,axis=1,inplace=True)
    except:
        pass

In [9]:
df.head(2)

,make_model,short_description,body_type,price,vat,km,registration,kW,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Gearing Type,Displacement,Cylinders,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,Comfort & Convenience,Entertainment & Media,Extras,Safety & Security,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from
0,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n2\n,"[\n06/2021\n, \n99 g CO2/km (comb)\n]","[\nYes\n, \nEuro 6\n]","[\n, \n, \n4 (Green)\n]","[\n, \n]","[\n, \n]",[],\nAudi\n,"[\n, A1, \n]","[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nMythosschwarz\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n5\n],[\n5\n],"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Armrest, Automatic climate ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Catalytic Converter, Voice Cont...","[ABS, Central door lock, Daytime running light...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,NaN,141 kW,"[, Used, , Gasoline]",NaN,NaN,NaN,NaN,NaN,NaN,[],\nAudi\n,"[\n, A1, \n]","[\n, 2017, \n]","[\n, Red, \n]",NaN,NaN,"[\nCloth, Grey\n]","[\n, Sedans, \n]",[\n3\n],[\n4\n],"[\n, Automatic, \n]","[\n1,798 cc\n]",[\n4\n],[\nfront\n],"[\n, Gasoline, \n]","[[5.6 l/100 km (comb)], [7.1 l/100 km (city)],...",[\n129 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Automatic climate control, ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Sport seats, Sport suspension, ...","[ABS, Central door lock, Central door lock wit...",[\n4 (Green)\n],[\n7\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def list_to_feat_cols2(df,col_list):
    dftemp = pd.DataFrame()
    for col in col_list:
        #max_list = max(df.loc[df[col].apply(lambda x : isinstance(x, list))][col], key=len)
        a = df[col].explode().unique()
        max_list = sorted([i for i in a if not pd.isnull(list(a)[list(a).index(i)])])
        for i in range(len(max_list)):
            dftemp[f"{col} {str(i+1)} {max_list[i]}"] = df[col].apply(lambda x : True if (type(x) == list and max_list[i] in x) else False) 
        df = df.drop(col,axis=1)
    df_added = pd.concat([df, dftemp], axis=1, sort=False)
    return df_added

In [11]:
col_list = ['Comfort & Convenience',"Safety & Security","Extras","Entertainment & Media","Type"]

In [12]:
list_to_feat_cols2(df,col_list)

,make_model,short_description,body_type,price,vat,km,registration,kW,hp,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Gearing Type,Displacement,Cylinders,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from,Comfort & Convenience 1 Air conditioning,Comfort & Convenience 2 Air suspension,Comfort & Convenience 3 Armrest,Comfort & Convenience 4 Automatic climate control,Comfort & Convenience 5 Auxiliary heating,Comfort & Convenience 6 Cruise control,Comfort & Convenience 7 Electric Starter,Comfort & Convenience 8 Electric tailgate,Comfort & Convenience 9 Electrical side mirrors,Comfort & Convenience 10 Electrically adjustable seats,Comfort & Convenience 11 Electrically heated windshield,Comfort & Convenience 12 Heads-up display,Comfort & Convenience 13 Heated steering wheel,Comfort & Convenience 14 Hill Holder,Comfort & Convenience 15 Keyless central door lock,Comfort & Convenience 16 Leather seats,Comfort & Convenience 17 Leather steering wheel,Comfort & Convenience 18 Light sensor,Comfort & Convenience 19 Lumbar support,Comfort & Convenience 20 Massage seats,Comfort & Convenience 21 Multi-function steering wheel,Comfort & Convenience 22 Navigation system,Comfort & Convenience 23 Panorama roof,Comfort & Convenience 24 Park Distance Control,Comfort & Convenience 25 Parking assist system camera,Comfort & Convenience 26 Parking assist system self-steering,Comfort & Convenience 27 Parking assist system sensors front,Comfort & Convenience 28 Parking assist system sensors rear,Comfort & Convenience 29 Power windows,Comfort & Convenience 30 Rain sensor,Comfort & Convenience 31 Seat heating,Comfort & Convenience 32 Seat ventilation,Comfort & Convenience 33 Split rear seats,Comfort & Convenience 34 Start-stop system,Comfort & Convenience 35 Sunroof,Comfort & Convenience 36 Tinted windows,Comfort & Convenience 37 Wind deflector,Comfort & Convenience 38 Windshield,Safety & Security 1 ABS,Safety & Security 2 Adaptive Cruise Control,Safety & Security 3 Adaptive headlights,Safety & Security 4 Alarm system,Safety & Security 5 Blind spot monitor,Safety & Security 6 Central door lock,Safety & Security 7 Central door lock with remote control,Safety & Security 8 Daytime running lights,Safety & Security 9 Driver drowsiness detection,Safety & Security 10 Driver-side airbag,Safety & Security 11 Electronic stability control,Safety & Security 12 Emergency brake assistant,Safety & Security 13 Emergency system,Safety & Security 14 Fog lights,Safety & Security 15 Head airbag,Safety & Security 16 Immobilizer,Safety & Security 17 Isofix,Safety & Security 18 LED Daytime Running Lights,Safety & Security 19 LED Headlights,Safety & Security 20 Lane departure warning system,Safety & Security 21 Night view assist,Safety & Security 22 Passenger-side airbag,Safety & Security 23 Power steering,Safety & Security 24 Rear airbag,Safety & Security 25 Side airbag,Safety & Security 26 Tire pressure monitoring system,Safety & Security 27 Traction control,Safety & Security 28 Traffic sign recognition,Safety & Security 29 Xenon headlights,Extras 1 Alloy wheels,Extras 2 Cab or rented Car,Extras 3 Catalytic Converter,Extras 4 Handicapped enabled,Extras 5 Right hand drive,Extras 6 Roof rack,Extras 7 Shift paddles,Extras 8 Ski bag,Extras 9 Sliding door,Extras 10 Sport package,Extras 11 Sport seats,Extras 12 Sport suspension,Extras 13 Touch screen,Extras 14 Trailer hitch,Extras 15 Tuned car,Extras 16 Voice Control,Extras 17 Winter tyres,Entertainment & Media 1 Bluetooth,Entertainment & Media 2 CD player,Entertainment & Media 3 Digital radio,Entertainment & Media 4 Hands-free equipment,Entertainment & Media 5 MP3,Entertainment & Media 6 On-board computer,Entertainment

In [13]:
#df5=pd.DataFrame({"ss":[['Air conditioning', 'Power windows'],['Air conditioning', 'sunroof'],np.nan]})

In [14]:
def list_to_feat_cols3(df,col_list):
    dftemp = pd.DataFrame()
    for col in col_list:
        #max_list = max(df.loc[df[col].apply(lambda x : isinstance(x, list))][col], key=len)
        a = df[col].explode().unique()
        max_list = sorted([i for i in a if not pd.isnull(list(a)[list(a).index(i)])])
        for i in range(len(max_list)):
            dftemp[f"{col} {str(i+1)} {max_list[i]}"] = df[col].apply(lambda x : True if (type(x) == list and max_list[i] in x) \
               else (False if (type(x) == list and max_list[i] not in x) else x ))
        df = df.drop(col,axis=1)
    df_added = pd.concat([df, dftemp], axis=1, sort=False)
    return df_added

In [15]:
df.head()

,make_model,short_description,body_type,price,vat,km,registration,kW,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Gearing Type,Displacement,Cylinders,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,Comfort & Convenience,Entertainment & Media,Extras,Safety & Security,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from
0,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n2\n,"[\n06/2021\n, \n99 g CO2/km (comb)\n]","[\nYes\n, \nEuro 6\n]","[\n, \n, \n4 (Green)\n]","[\n, \n]","[\n, \n]",[],\nAudi\n,"[\n, A1, \n]","[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nMythosschwarz\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n5\n],[\n5\n],"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Armrest, Automatic climate ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Catalytic Converter, Voice Cont...","[ABS, Central door lock, Daytime running light...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,NaN,141 kW,"[, Used, , Gasoline]",NaN,NaN,NaN,NaN,NaN,NaN,[],\nAudi\n,"[\n, A1, \n]","[\n, 2017, \n]","[\n, Red, \n]",NaN,NaN,"[\nCloth, Grey\n]","[\n, Sedans, \n]",[\n3\n],[\n4\n],"[\n, Automatic, \n]","[\n1,798 cc\n]",[\n4\n],[\nfront\n],"[\n, Gasoline, \n]","[[5.6 l/100 km (comb)], [7.1 l/100 km (city)],...",[\n129 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Automatic climate control, ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Sport seats, Sport suspension, ...","[ABS, Central door lock, Central door lock wit...",[\n4 (Green)\n],[\n7\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",02/2016,NaN,85 kW,"[, Used, , Diesel (Particulate Filter)]",\n1\n,NaN,NaN,"[\n, \n, \n99 g CO2/km (comb)\n]",NaN,NaN,[],\nAudi\n,"[\n, A1, \n]","[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nmythosschwarz metallic\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n4\n],[\n4\n],"[\n, Automatic, \n]","[\n1,598 cc\n]",NaN,[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.4 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Cruise control, Electrical ...","[MP3, On-board computer]","[Alloy wheels, Voice Control]","[ABS, Central door lock, Daytime running light...",[\n4 (Green)\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Audi A1,1.4 TDi Design S tronic,Sedans,14500,None,"73,000 km",08/2016,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n1\n,NaN,NaN,NaN,"[\n, \n, \n99 g CO2/km (comb)\n]","[\n, \n, \nEuro 6\n]",[],\nAudi\n,"[\n, A1, \n]","[\n, 2016, \n]","[\n, Brown, \n]",[\nMetallic\n],NaN,NaN,"[\n, Sedans, \n]",[\n3\n],[\n4\n],"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],NaN,"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air suspension, Armrest, Auxiliary heating, E...","[Bluetooth, CD player, Hands-free equipment, M...","[Alloy wheels, Sport seats, Voice Control]","[ABS, Alarm system, Central door lock with rem...",NaN,[\n6\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Audi A1,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790,None,"16,200 km",05/2016,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n1\n,NaN,"[\nYes\n, \n109 g CO2/km (comb)\n]","[\n, \n, \nEuro 6\n]","[\n, \n, \n4 (Green)\n]","[\n, \n]",[],\nAudi\n,"[\n, A1, \n]","[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\n

In [16]:
def fill_most(df, group_col, col_name):
    for group in list(df[group_col].unique()):
        grp_inx = list(df[df[group_col]==group][col_name].index)
        df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[df[group_col]==group][col_name].mode()[0])

In [17]:
# now we will convert lists to columns, after removing "description" since its just a comment for car and does not affect the price.
list_columns, nonlist_columns = check_is_list(df)
for i in list_columns:
    df = list_to_columns(df,i)

In [18]:
df_orj = df

In [19]:
df['Displacement'] = df['Displacement'].apply(lambda x : x if pd.isna(x) else x.split("cc")[0].strip())
df.rename(columns={'Displacement': 'Displacement (cc)'},inplace=True)
df["Displacement (cc)"] = df["Displacement (cc)"].apply(lambda x : round(int(x.replace(",",""))/1000,1) if type(x)==str else x)


In [20]:
#replace all "", [], \n, values to np.Nan or delete if possible in df
for i in df.columns:
    df[i] = df[i].replace("",np.NaN)
    df[i] = df[i].apply(lambda x: np.nan if (type(x) == list and len(x) == 0) else x) # convert [] to np.NAN
    try:
        df[i] = df[i].apply(lambda x: list(map(lambda x: x.strip(), x)) if type(x)==list else x)
    except:
        continue
    df[i] = df[i].apply(lambda x : np.NaN if x=='\n' else x)
    df[i] = df[i].apply(lambda x: (("".join(x.replace("\n",""))) if (type(x)==str and type(x) != float) else x) if x!=np.nan else x)

In [172]:
# Fills missing values with combination of columns, Follows priority 
#firstly tries 3 column combination and fills nulls and applies remaining nulls with 2 colum combination and so on.
def fill_most_with1_temp(df,target_col,ref_col1):
    df_temp = df[[ref_col1,target_col]]
    for group1 in list(df_temp[ref_col1].unique()):
        grp_inx = list(df_temp[(df_temp[ref_col1]==group1)][target_col].index)
        try:
            fill_value = df_temp[(df_temp[ref_col1] == group1)][target_col].mode()[0]
        except:
            fill_value = 'NaN'
            continue
        df_temp[target_col].iloc[grp_inx] = df_temp[target_col].iloc[grp_inx].fillna(fill_value)
    return df_temp
def fill_most_with1_perm(df,target_col,ref_col1):
    for group1 in list(df[ref_col1].unique()):
        grp_inx = list(df[(df[ref_col1]==group1)][target_col].index)
        try:
            fill_value = df[(df[ref_col1] == group1)][target_col].mode()[0]
        except:
            fill_value = 'NaN'
            continue
        df[target_col].iloc[grp_inx] = df[target_col].iloc[grp_inx].fillna(fill_value)
    return df
def fill_most_with2_temp(df,target_col,col1, col2):
    df_temp = df[[col1,col2,target_col]]
    for group1 in list(df_temp[col1].unique()):
        for group2 in list(df_temp[df_temp[col1]==group1][col2].unique()):
            grp_inx = list(df_temp[(df_temp[col1]==group1) & (df_temp[col2] == group2)][target_col].index)
            try:
                fill_value = df_temp[(df_temp[col1] == group1) & (df_temp[col2] == group2)][target_col].mode()[0]
            except:
                fill_value = 'NaN'
                continue
            df_temp[target_col].iloc[grp_inx] = df_temp[target_col].iloc[grp_inx].fillna(fill_value)  
    return df_temp
def fill_most_with2_perm(df,target_col, col1, col2):
    for group1 in list(df[col1].unique()):
        for group2 in list(df[df[col1]==group1][col2].unique()):
            grp_inx = list(df[(df[col1]==group1) & (df[col2] == group2)][target_col].index)
            try:
                fill_value = df[(df[col1] == group1) & (df[col2] == group2)][target_col].mode()[0]
            except:
                fill_value = 'NaN'
                continue
            df[target_col].iloc[grp_inx] = df[target_col].iloc[grp_inx].fillna(fill_value)  
    return df
#df[(df["Make"] == "Audi") & (df["Model2"] == "A3")  & (df["Displacement (cc)"] == 1.5)]["body_type"].mode()[0]
def fill_most_with3_temp(df,target_col,col1, col2, col3):
    df_temp = df[[col1,col2,col3,target_col]]
    for group1 in list(df_temp[col1].unique()):
        for group2 in list(df_temp[df_temp[col1]==group1][col2].unique()):
            for group3 in list(df_temp[(df_temp[col1]==group1) & (df_temp[col2]==group2)][col3].unique()):
                grp_inx = list(df_temp[(df_temp[col1]==group1) & (df_temp[col2] == group2)& (df_temp[col3]==group3)][target_col].index)
                try:
                    fill_value = df_temp[(df_temp[col1] == group1) & (df_temp[col2] == group2) & (df_temp[col3] == group3)][target_col].mode()[0]
                except:
                    fill_value = 'NaN'
                    continue
                df_temp[target_col].iloc[grp_inx] = df_temp[target_col].iloc[grp_inx].fillna(fill_value)  
    return df_temp



def fill_most_with3_perm(df,target_col,col1, col2, col3):
    for group1 in list(df[col1].unique()):
        for group2 in list(df[df[col1]==group1][col2].unique()):
            for group3 in list(df[(df[col1]==group1) & (df[col2] == group2)][col3].unique()):
                grp_inx = list(df[(df[col1]==group1) & (df[col2] == group2)& (df[col3]==group3)][target_col].index)
                try:
                    fill_value = df[(df[col1] == group1) & (df[col2] == group2) & (df[col3] == group3)][target_col].mode()[0]
                except:
                    fill_value = 'NaN'
                    continue
                df[target_col].iloc[grp_inx] = df[target_col].iloc[grp_inx].fillna(fill_value)  
    return df

def fill_priority_perm(df,target_col,col1, col2, col3):
    fill_most_with3_perm(df,target_col,col1, col2, col3)
    fill_most_with2_perm(df,target_col,col1, col2)
    fill_most_with1_perm(df,target_col,col1)
    return df

def fill_priority_temp(df,target_col,col1, col2, col3):
    df_temp1 = fill_most_with3_temp(df,target_col,col1, col2, col3)
    df_temp2 = fill_most_with2_temp(df_temp1,target_col,col1, col2)
    df_temp3 = fill_most_with1_temp(df_temp2,target_col,col1)
    return df_temp3
   

In [169]:
df["Drive chain"].value_counts(dropna = False)

front    8886
NaN      6858
4WD       171
rear        4
Name: Drive chain, dtype: int64

In [170]:
fill_priority_perm(df,"Drive chain","make_model","Gearing Type2","Fuel2")

,make_model,short_description,body_type,price,vat,km,registration,kW,hp,Make,Availability,Available from,Type1,Type2,Type3,Type4,Previous Owners1,Previous Owners2,Previous Owners3,Previous Owners4,Previous Owners5,Previous Owners6,Previous Owners7,Previous Owners8,Next Inspection1,Next Inspection2,Next Inspection3,Next Inspection4,Next Inspection5,Next Inspection6,Next Inspection7,Next Inspection8,Inspection new1,Inspection new2,Inspection new3,Inspection new4,Inspection new5,Inspection new6,Inspection new7,Inspection new8,Warranty1,Warranty2,Warranty3,Warranty4,Warranty5,Warranty6,Warranty7,Warranty8,Warranty9,Full Service1,Full Service2,Full Service3,Full Service4,Full Service5,Full Service6,Full Service7,Full Service8,Full Service9,Non-smoking Vehicle1,Non-smoking Vehicle2,Non-smoking Vehicle3,Non-smoking Vehicle4,Non-smoking Vehicle5,Non-smoking Vehicle6,Non-smoking Vehicle7,Non-smoking Vehicle8,Non-smoking Vehicle9,Model1,Model2,Model3,First Registration1,First Registration2,First Registration3,Body Color1,Body Color2,Body Color3,Paint Type,Body Color Original,Upholstery,Body1,Body2,Body3,Nr. of Doors,Nr. of Seats,Gearing Type1,Gearing Type2,Gearing Type3,Displacement (cc),Cylinders,Drive chain,Fuel1,Fuel2,Fuel3,Consumption1,Consumption2,Consumption3,Consumption4,Consumption5,Consumption6,Consumption7,CO2 Emission1,CO2 Emission2,CO2 Emission3,Emission Class1,Emission Class2,Emission Class3,Comfort & Convenience1,Comfort & Convenience2,Comfort & Convenience3,Comfort & Convenience4,Comfort & Convenience5,Comfort & Convenience6,Comfort & Convenience7,Comfort & Convenience8,Comfort & Convenience9,Comfort & Convenience10,Comfort & Convenience11,Comfort & Convenience12,Comfort & Convenience13,Comfort & Convenience14,Comfort & Convenience15,Comfort & Convenience16,Comfort & Convenience17,Comfort & Convenience18,Comfort & Convenience19,Comfort & Convenience20,Comfort & Convenience21,Comfort & Convenience22,Comfort & Convenience23,Comfort & Convenience24,Comfort & Convenience25,Comfort & Convenience26,Comfort & Convenience27,Comfort & Convenience28,Comfort & Convenience29,Comfort & Convenience30,Comfort & Convenience31,Comfort & Convenience32,Comfort & Convenience33,Entertainment & Media1,Entertainment & Media2,Entertainment & Media3,Entertainment & Media4,Entertainment & Media5,Entertainment & Media6,Entertainment & Media7,Entertainment & Media8,Entertainment & Media9,Entertainment & Media10,Extras1,Extras2,Extras3,Extras4,Extras5,Extras6,Extras7,Extras8,Extras9,Extras10,Safety & Security1,Safety & Security2,Safety & Security3,Safety & Security4,Safety & Security5,Safety & Security6,Safety & Security7,Safety & Security8,Safety & Security9,Safety & Security10,Safety & Security11,Safety & Security12,Safety & Security13,Safety & Security14,Safety & Security15,Safety & Security16,Safety & Security17,Safety & Security18,Safety & Security19,Safety & Security20,Safety & Security21,Safety & Security22,Safety & Security23,Safety & Security24,Safety & Security25,Safety & Security26,Safety & Security27,Safety & Security28,Safety & Security29,Emission Label1,Emission Label2,Emission Label3,Gears,Country version,Electricity consumption,Last Service Date1,Last Service Date2,Last Service Date3,Last Service Date4,Last Service Date5,Last Service Date6,Last Service Date7,Last Service Date8,Other Fuel Types1,Other Fuel Types2,Other Fuel Types3,Last Timing Belt Service Date1,Last Timing Belt Service Date2
0,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,NaN,66 kW,Audi,NaN,NaN,NaN,Used,NaN,Diesel (Particulate Filter),2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/2021,99 g CO2/km (comb),NaN,NaN,NaN,NaN,NaN,NaN,Yes,Euro 6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 (Green),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1,NaN,NaN,2016,NaN,NaN,Black,NaN,Metallic,Mythosschwarz,"Cloth, Black",NaN,Sedans,NaN,5,5,NaN,Automatic,NaN,1.4,3,front,NaN,Diesel (Particulate Filter),N

In [171]:
df["Drive chain"].value_counts(dropna = False)

front    15712
4WD        203
rear         4
Name: Drive chain, dtype: int64

In [167]:
#df7 = fill_most_with3_temp(df,"Drive chain","make_model","Gearing Type2","Fuel2")

In [159]:
df7["Drive chain"].value_counts(dropna = False)

front    15514
NaN        206
4WD        195
rear         4
Name: Drive chain, dtype: int64

In [160]:
# df2["Drive chain"].value_counts(dropna = False)

In [ ]:
fill_most_with2_temp

In [154]:
df7.isnull().sum()

make_model       0
Gearing Type2    0
Drive chain      8
dtype: int64

In [140]:
target_col = "Drive chain"
col1 = "make_model"
col2 = "Gearing Type2"
df_temp = df[[col1,col2,target_col]]
for group1 in list(df_temp[col1].unique()):
    for group2 in list(df_temp[df_temp[col1]==group1][col2].unique()):
        grp_inx = list(df_temp[(df_temp[col1]==group1) & (df_temp[col2] == group2)][target_col].index)
        #print(grp_inx)
        print(group1,group2)
        try:
            fill_value = df_temp[(df_temp[col1] == group1) & (df_temp[col2] == group2)][target_col].mode()[0]
        except:
            fill_value = 'NaN'
            continue
        #print(group1,group2,fill_value)
        df_temp[target_col].iloc[grp_inx] = df_temp[target_col].iloc[grp_inx].fillna(fill_value)

Audi A1 Automatic
Audi A1 Manual
Audi A1 Semi-automatic
Audi A2 Manual
Audi A3 Automatic
Audi A3 Manual
Audi A3 Semi-automatic
Opel Astra Automatic
Opel Astra Manual
Opel Astra Semi-automatic
Opel Corsa Automatic
Opel Corsa Manual
Opel Corsa Semi-automatic
Opel Insignia Automatic
Opel Insignia Manual
Renault Clio Automatic
Renault Clio Manual
Renault Clio Semi-automatic
Renault Duster Automatic
Renault Duster Manual
Renault Espace Automatic
Renault Espace Manual
Renault Espace Semi-automatic


In [147]:
df4.isnull().sum()

make_model       0
Gearing Type2    0
Drive chain      8
dtype: int64

In [142]:
pd.options.display.max_rows = 16

df_temp.loc[[14894, 14895, 14896, 14897, 14898, 14899, 14900, 14901]]

,make_model,Gearing Type2,Drive chain
14894,Renault Duster,Automatic,NaN
14895,Renault Duster,Automatic,NaN
14896,Renault Duster,Automatic,NaN
14897,Renault Duster,Automatic,NaN
14898,Renault Duster,Automatic,NaN
14899,Renault Duster,Automatic,NaN
14900,Renault Duster,Automatic,NaN
14901,Renault Duster,Automatic,NaN


In [148]:
df4["Drive chain"].value_counts(dropna = False)

front    15712
4WD        195
NaN          8
rear         4
Name: Drive chain, dtype: int64

In [149]:
df["Drive chain"].value_counts(dropna = False)

front    8886
NaN      6858
4WD       171
rear        4
Name: Drive chain, dtype: int64

In [163]:
pd.options.display.max_rows = 200

df.groupby(["make_model","Gearing Type2","Fuel2"])["Drive chain"].describe()

count unique  \
make_model     Gearing Type2  Fuel2                                        
Audi A1        Automatic      Diesel                           76      2   
                              Diesel (Particulate Filter)     106      1   
                              Gasoline                        172      1   
                              Gasoline (Particulate Filter)     0      0   
                              Regular/Benzine 91               67      1   
...                                                           ...    ...   
Renault Espace Manual         Super 95 (Particulate Filter)     1      1   
               Semi-automatic Diesel                            5      1   
                              Diesel (Particulate Filter)       4      2   
                              Gasoline                          0      0   
                              Regular/Benzine 91                1      1   

                                                               top freq  
make_model     Gearing Type2  Fuel2                                      
Audi A1        Automatic      Diesel                         front   75  
                              Diesel (Particulate Filter)    front  106  
                              Gasoline                       front  172  
                              Gasoline (Particulate Filter)    NaN  NaN  
                              Regular/Benzine 91             front   67  
...                                                            ...  ...  
Renault Espace Manual         Super 95 (Particulate Filter)  front    1  
               Semi-automatic Diesel                         front    5  
                              Diesel (Particulate Filter)    front    3  
                              Gasoline                         NaN  NaN  
                              Regular/Benzine 91             front    1  

[314 rows x 4 columns]

In [85]:
df.isnull().sum().sort_values()

make_model           0
Gearing Type2        0
Fuel2                0
                 ...  
Body Color1      15919
Body1            15919
Fuel1            15919
Length: 207, dtype: int64

In [88]:
df["Gearing Type2"].value_counts(dropna = False)

Manual            8153
Automatic         7297
Semi-automatic     469
Name: Gearing Type2, dtype: int64

In [31]:
#func_spec(df,'body_type','Make', 'Model2', 'Displacement (cc)')

In [32]:
df[(df["Make"] == "Audi") & (df["Model2"] == "A3")  ][["body_type","Displacement (cc)"]]
df5 = pd.DataFrame({"Make":["Audi","Audi"],'Model2':["A3","A3"],"Displacement (cc)":[1.5,1.5],"body_type":["Sedans",np.nan]})

In [33]:
df5

,Make,Model2,Displacement (cc),body_type
0,Audi,A3,1.5,Sedans
1,Audi,A3,1.5,NaN


In [34]:
#func_spec(df5,'body_type','Make', 'Model2', 'Displacement (cc)')

In [35]:
df6 = df.head(142) #143de hata veriyor.

In [36]:
df.loc[[142,143]]

,make_model,short_description,body_type,price,vat,km,registration,kW,hp,Make,Availability,Available from,Type1,Type2,Type3,Type4,Previous Owners1,Previous Owners2,Previous Owners3,Previous Owners4,Previous Owners5,Previous Owners6,Previous Owners7,Previous Owners8,Next Inspection1,Next Inspection2,Next Inspection3,Next Inspection4,Next Inspection5,Next Inspection6,Next Inspection7,Next Inspection8,Inspection new1,Inspection new2,Inspection new3,Inspection new4,Inspection new5,Inspection new6,Inspection new7,Inspection new8,Warranty1,Warranty2,Warranty3,Warranty4,Warranty5,Warranty6,Warranty7,Warranty8,Warranty9,Full Service1,Full Service2,Full Service3,Full Service4,Full Service5,Full Service6,Full Service7,Full Service8,Full Service9,Non-smoking Vehicle1,Non-smoking Vehicle2,Non-smoking Vehicle3,Non-smoking Vehicle4,Non-smoking Vehicle5,Non-smoking Vehicle6,Non-smoking Vehicle7,Non-smoking Vehicle8,Non-smoking Vehicle9,Model1,Model2,Model3,First Registration1,First Registration2,First Registration3,Body Color1,Body Color2,Body Color3,Paint Type,Body Color Original,Upholstery,Body1,Body2,Body3,Nr. of Doors,Nr. of Seats,Gearing Type1,Gearing Type2,Gearing Type3,Displacement (cc),Cylinders,Drive chain,Fuel1,Fuel2,Fuel3,Consumption1,Consumption2,Consumption3,Consumption4,Consumption5,Consumption6,Consumption7,CO2 Emission1,CO2 Emission2,CO2 Emission3,Emission Class1,Emission Class2,Emission Class3,Comfort & Convenience1,Comfort & Convenience2,Comfort & Convenience3,Comfort & Convenience4,Comfort & Convenience5,Comfort & Convenience6,Comfort & Convenience7,Comfort & Convenience8,Comfort & Convenience9,Comfort & Convenience10,Comfort & Convenience11,Comfort & Convenience12,Comfort & Convenience13,Comfort & Convenience14,Comfort & Convenience15,Comfort & Convenience16,Comfort & Convenience17,Comfort & Convenience18,Comfort & Convenience19,Comfort & Convenience20,Comfort & Convenience21,Comfort & Convenience22,Comfort & Convenience23,Comfort & Convenience24,Comfort & Convenience25,Comfort & Convenience26,Comfort & Convenience27,Comfort & Convenience28,Comfort & Convenience29,Comfort & Convenience30,Comfort & Convenience31,Comfort & Convenience32,Comfort & Convenience33,Entertainment & Media1,Entertainment & Media2,Entertainment & Media3,Entertainment & Media4,Entertainment & Media5,Entertainment & Media6,Entertainment & Media7,Entertainment & Media8,Entertainment & Media9,Entertainment & Media10,Extras1,Extras2,Extras3,Extras4,Extras5,Extras6,Extras7,Extras8,Extras9,Extras10,Safety & Security1,Safety & Security2,Safety & Security3,Safety & Security4,Safety & Security5,Safety & Security6,Safety & Security7,Safety & Security8,Safety & Security9,Safety & Security10,Safety & Security11,Safety & Security12,Safety & Security13,Safety & Security14,Safety & Security15,Safety & Security16,Safety & Security17,Safety & Security18,Safety & Security19,Safety & Security20,Safety & Security21,Safety & Security22,Safety & Security23,Safety & Security24,Safety & Security25,Safety & Security26,Safety & Security27,Safety & Security28,Safety & Security29,Emission Label1,Emission Label2,Emission Label3,Gears,Country version,Electricity consumption,Last Service Date1,Last Service Date2,Last Service Date3,Last Service Date4,Last Service Date5,Last Service Date6,Last Service Date7,Last Service Date8,Other Fuel Types1,Other Fuel Types2,Other Fuel Types3,Last Timing Belt Service Date1,Last Timing Belt Service Date2
142,Audi A1,1.6 TDI METAL PLUS *S-TRONIC* BI-COLOR*NAVI_ X...,Sedans,18200,VAT deductible,"16,500 km",01/2018,NaN,85 kW,Audi,NaN,NaN,NaN,Used,NaN,Diesel (Particulate Filter),1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12 months,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1,NaN,NaN,2018,NaN,NaN,Black,NaN,Metallic,NaN,"Cloth, Black",NaN,Sedans,NaN,5,5,NaN,Automatic,NaN,NaN,4,front,NaN,Diesel (Particulate Filter),NaN,NaN,NaN,NaN,NaN,NaN,N

In [37]:
#func_spec(df6,'body_type','Make', 'Model2', 'Displacement (cc)')

In [38]:
df[pd.isna(df['body_type'])]['body_type']

3175     None
3255     None
3975     None
         ... 
15712    None
15736    None
15770    None
Name: body_type, Length: 60, dtype: object

In [39]:
df7 = df[['body_type','Make', 'Model2', 'Displacement (cc)']].loc[[142,143]]
df7

,body_type,Make,Model2,Displacement (cc)
142,Sedans,Audi,A1,NaN
143,Compact,Audi,A1,1.6


In [40]:
#func_spec(df7,'body_type','Make', 'Model2', 'Displacement (cc)')

In [155]:
df.Fuel2

0                          Diesel (Particulate Filter)
1                                             Gasoline
2                          Diesel (Particulate Filter)
3                          Diesel (Particulate Filter)
4                          Diesel (Particulate Filter)
                             ...                      
15914                      Diesel (Particulate Filter)
15915    Super 95 / Super Plus 98 (Particulate Filter)
15916                                           Diesel
15917                                           Diesel
15918                                         Super 95
Name: Fuel2, Length: 15919, dtype: object

In [ ]:
df["make_model"].isnull().sum()

In [174]:
df.head(2)

,make_model,short_description,body_type,price,vat,km,registration,kW,hp,Make,Availability,Available from,Type1,Type2,Type3,Type4,Previous Owners1,Previous Owners2,Previous Owners3,Previous Owners4,Previous Owners5,Previous Owners6,Previous Owners7,Previous Owners8,Next Inspection1,Next Inspection2,Next Inspection3,Next Inspection4,Next Inspection5,Next Inspection6,Next Inspection7,Next Inspection8,Inspection new1,Inspection new2,Inspection new3,Inspection new4,Inspection new5,Inspection new6,Inspection new7,Inspection new8,Warranty1,Warranty2,Warranty3,Warranty4,Warranty5,Warranty6,Warranty7,Warranty8,Warranty9,Full Service1,Full Service2,Full Service3,Full Service4,Full Service5,Full Service6,Full Service7,Full Service8,Full Service9,Non-smoking Vehicle1,Non-smoking Vehicle2,Non-smoking Vehicle3,Non-smoking Vehicle4,Non-smoking Vehicle5,Non-smoking Vehicle6,Non-smoking Vehicle7,Non-smoking Vehicle8,Non-smoking Vehicle9,Model1,Model2,Model3,First Registration1,First Registration2,First Registration3,Body Color1,Body Color2,Body Color3,Paint Type,Body Color Original,Upholstery,Body1,Body2,Body3,Nr. of Doors,Nr. of Seats,Gearing Type1,Gearing Type2,Gearing Type3,Displacement (cc),Cylinders,Drive chain,Fuel1,Fuel2,Fuel3,Consumption1,Consumption2,Consumption3,Consumption4,Consumption5,Consumption6,Consumption7,CO2 Emission1,CO2 Emission2,CO2 Emission3,Emission Class1,Emission Class2,Emission Class3,Comfort & Convenience1,Comfort & Convenience2,Comfort & Convenience3,Comfort & Convenience4,Comfort & Convenience5,Comfort & Convenience6,Comfort & Convenience7,Comfort & Convenience8,Comfort & Convenience9,Comfort & Convenience10,Comfort & Convenience11,Comfort & Convenience12,Comfort & Convenience13,Comfort & Convenience14,Comfort & Convenience15,Comfort & Convenience16,Comfort & Convenience17,Comfort & Convenience18,Comfort & Convenience19,Comfort & Convenience20,Comfort & Convenience21,Comfort & Convenience22,Comfort & Convenience23,Comfort & Convenience24,Comfort & Convenience25,Comfort & Convenience26,Comfort & Convenience27,Comfort & Convenience28,Comfort & Convenience29,Comfort & Convenience30,Comfort & Convenience31,Comfort & Convenience32,Comfort & Convenience33,Entertainment & Media1,Entertainment & Media2,Entertainment & Media3,Entertainment & Media4,Entertainment & Media5,Entertainment & Media6,Entertainment & Media7,Entertainment & Media8,Entertainment & Media9,Entertainment & Media10,Extras1,Extras2,Extras3,Extras4,Extras5,Extras6,Extras7,Extras8,Extras9,Extras10,Safety & Security1,Safety & Security2,Safety & Security3,Safety & Security4,Safety & Security5,Safety & Security6,Safety & Security7,Safety & Security8,Safety & Security9,Safety & Security10,Safety & Security11,Safety & Security12,Safety & Security13,Safety & Security14,Safety & Security15,Safety & Security16,Safety & Security17,Safety & Security18,Safety & Security19,Safety & Security20,Safety & Security21,Safety & Security22,Safety & Security23,Safety & Security24,Safety & Security25,Safety & Security26,Safety & Security27,Safety & Security28,Safety & Security29,Emission Label1,Emission Label2,Emission Label3,Gears,Country version,Electricity consumption,Last Service Date1,Last Service Date2,Last Service Date3,Last Service Date4,Last Service Date5,Last Service Date6,Last Service Date7,Last Service Date8,Other Fuel Types1,Other Fuel Types2,Other Fuel Types3,Last Timing Belt Service Date1,Last Timing Belt Service Date2
0,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,NaN,66 kW,Audi,NaN,NaN,NaN,Used,NaN,Diesel (Particulate Filter),2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/2021,99 g CO2/km (comb),NaN,NaN,NaN,NaN,NaN,NaN,Yes,Euro 6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 (Green),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1,NaN,NaN,2016,NaN,NaN,Black,NaN,Metallic,Mythosschwarz,"Cloth, Black",NaN,Sedans,NaN,5,5,NaN,Automatic,NaN,1.4,3,front,NaN,Diesel (Particulate Filter),N

In [173]:
df['body_type'].isnull().sum()

60

In [ ]:
fill_priority_temp(df,'body_type',"make_model", "First Registration2", col3)